In [1]:
import numpy as np
import databricks.koalas as ks
from pyspark.sql import SparkSession
import os
#import sys
from scipy.sparse import csr_matrix
import geopandas as gpd
import pickle
from pathlib import Path
import argparse
from pyspark.sql.functions import expr, to_timestamp, col, unix_timestamp, first, sum, count
from pyspark.sql.types import *
from pyspark.sql.types import LongType, IntegerType
from pyspark.sql.window import Window

In [2]:
# Datapath file CSV (dataset)
pathCSV = 'data/trips2323_new.csv'

# Path ShapeFile
pathShapeF = 'data/shapefile/shapefile_2323.shp'

# Path for results
pathRes = 'results/'

# Skip division per 0
np.seterr(divide='ignore', invalid='ignore')



def preprocessing(df):
    #df_cache = df.spark.cache()
    df = df.rename(columns={'from_zone':'from_zone_fid', 'from_timedate':'from_timedate_gmt', 'to_zone':'to_zone_fid', 'to_timedate':'to_timedate_gmt'})

    df = df.dropna(subset=['to_zone_fid'])
    fill = {'from_zone_fid': -1}
    df = df.fillna(value=fill)


    # Initial Filter
    # (df.triptime_s < 3278)
    df = df.loc[ (df.tripdistance_m > 0) & (df.stoptime_s > 0)]# & (df.stoptime_s < 604800)]

    #df = df.sort_values(by=['from_timedate_gmt', 'to_timedate_gmt'])
    #df = df.reset_index(drop = True)

    df_spark = df.to_spark()

    df_spark = df_spark.withColumn("from_timedate_gmt", expr("substring(from_timedate_gmt, 1, length(from_timedate_gmt)-3)"))
    df_spark = df_spark.withColumn("to_timedate_gmt", expr("substring(to_timedate_gmt, 1, length(to_timedate_gmt)-3)"))
    #df_spark = df_spark.withColumn("from_timedate_gmt", df_spark.from_timedate_gmt.substr(1, 6))

    df = df_spark.to_koalas()

    #df_cache.spark.unpersist()

    return df

def square(s):
    return s/3600


In [3]:
from pyspark.sql.functions import expr, to_timestamp, col, unix_timestamp, first, sum, count

# Add 'totimedate_plus_stoptime' column calculated by making the sum of to_date and the stoptime
def add_new_stoptime_column(df):
    #df_cache = df.spark.cache()
    df_spark = df.to_spark()
    
    df_spark = df_spark.withColumn('new_stoptime', col('stoptime_s') + 60 - (col('stoptime_s') % 60))\
                       .withColumn('totimedate_plus_stoptime', (unix_timestamp(col('to_timedate_gmt').cast(TimestampType())) + col('new_stoptime')))\
                       .withColumn('totimedate_plus_stoptime', to_timestamp(col('totimedate_plus_stoptime')))

    df_spark = df_spark.drop('new_stoptime')

    df = df_spark.to_koalas()

    return df

In [4]:
spark = SparkSession.builder\
        .master("local[10]")\
        .appName("Pyspark")\
        .config('spark.driver.memory', "126g")\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.cores', "100")\
        .config("spark.local.dir", "/home/gpu2/spark-temp")\
        .getOrCreate()
spark.catalog.clearCache()
    
    
# Upload CSV datas
df = ks.read_csv(pathCSV, sep=';', dtype = {'from_timedate':str, 'to_timedate':str})
#df = df.spark.cache()
#df = df.spark.repartition(80)
df = preprocessing(df)
#   print(df.head())
#   exit()
print('-----------OUTPUT------------')
#df_cache = df.spark.cache()
df = df.rename(columns={'from_zone':'from_zone_fid', 'from_timedate':'from_timedate_gmt', 'to_zone':'to_zone_fid', 'to_timedate':'to_timedate_gmt'})
df = df.dropna(subset=['to_zone_fid'])
fill = {'from_zone_fid': -1}
df = df.fillna(value=fill)
# Iitial Filter
# (f.triptime_s < 3278)
df = df.loc[ (df.tripdistance_m > 0) & (df.stoptime_s > 0)] # & (df.stoptime_s != 31422298)]
df = df.sort_values(by=['idterm', 'to_timedate_gmt'])
casa = df[df.triptime_s>8500].head(2)
#df.stoptime_s = df.stoptime_s.apply(square)
#test = df.stoptime_s.to_numpy()

#ax = 
#df.stoptime_s.plot.kde(bw_method=3)  

22/06/15 12:03:39 WARN Utils: Your hostname, gpu2-Standard-PC-Q35-ICH9-2009 resolves to a loopback address: 127.0.1.1; using 10.0.0.205 instead (on interface enp6s18)
22/06/15 12:03:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/15 12:03:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/15 12:03:40 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


-----------OUTPUT------------


In [5]:
# Lower limit of times in the dataset
def time_limits_inf_2(df, intv):
    #df_cache = dataframe.spark.cache()

    # Generation of lower and upper bound of the calendar
    min = ks.to_datetime(df.from_timedate_gmt.min(), format='%Y/%m/%d %H:%M')  # Lower date in the df
    min = min.replace(minute=00)  # Set minutes to 00
    max = ks.to_datetime(df.to_timedate_gmt.max(), format='%Y/%m/%d %H:%M')  # Upper date in the df
    max = max.replace(hour=23, minute=59)  # Upper date set to 23:59

    # Create calendar
    cal = calendar(min, max, intv)
    cal = cal.to_frame(index=False, name='date')
    cal = cal.to_spark()
    
    #print(cal.show())
    df_spark = df.to_spark()
    
    df_spark.createOrReplaceTempView('df_spark')
    cal.createOrReplaceTempView('cal')
    res1 = df_spark.join(cal).where((to_date(df_spark.from_timedate_gmt) == to_date(cal.date)) & \
                               (df_spark.from_timedate_gmt.cast(TimestampType()) >= cal.date.cast(TimestampType()))).withColumnRenamed('date','date_1')#.crossJoin(cal).withColumnRenamed('date', 'date_2')#.crossJoin(cal).withColumnRenamed('date', 'date_3')

    res1 = res1.drop('from_timedate_gmt')#, 'to_timedate_gmt')#, 'date_2')
    res1 = res1.withColumnRenamed('date_1', 'from_timedate_gmt')
    w = Window.partitionBy("idtrajectory").orderBy(col("from_timedate_gmt").desc()) #, col("to_timedate_gmt").desc())
    res1 = res1.withColumn("row", row_number().over(w)).where("row == 1").drop("row")
    res1.createOrReplaceTempView('res1')
    
    res2 = res1.join(cal).where((to_date(df_spark.to_timedate_gmt) == to_date(cal.date)) & \
                               (df_spark.to_timedate_gmt.cast(TimestampType()) >= cal.date.cast(TimestampType()))).withColumnRenamed('date','date_1')
    
    res2 = res2.drop('to_timedate_gmt')#, 'to_timedate_gmt')#, 'date_2')
    res2 = res2.withColumnRenamed('date_1', 'to_timedate_gmt')
    w = Window.partitionBy("idtrajectory").orderBy(col("to_timedate_gmt").desc())
    res2 = res2.withColumn("row", row_number().over(w)).where("row == 1").drop("row")
    dataframe = res2.to_koalas()

    return dataframe

In [6]:
# Path ShapeFile
pathShapeF = 'data/shapefile/shapefile_2323.shp'

# Add 'weight' column calculated by making the quotient between the chosen
# interval and the delta of the time taken for the travel
def add_peso_column(dataframe, calendario):
    delta_cal = calendario.to_list()[1] - calendario.to_list()[0]
    delta_cal = delta_cal.seconds

    df_spark = dataframe.to_spark()
    
    df_spark = df_spark.withColumn('peso', col('to_timedate_gmt').cast(LongType()) - col('from_timedate_gmt').cast(LongType()))\
                       .withColumn('peso', delta_cal / (col('peso') + delta_cal))
    

    dataframe = df_spark.to_koalas()
    return dataframe

In [7]:
# Generator of calendar with specific timestamp (start/end='dd-mm-yyyy', freq='20min', '8hour', ...)
def calendar(start, end, freq):
    calendar = ks.date_range(start=start, end=end, freq=freq)
    return calendar

In [8]:
# ADJ matrix


from pyspark.sql.window import Window
from pyspark.sql.functions import concat_ws, row_number, date_format,from_unixtime,lag, col, lead, to_date, unix_timestamp, datediff, lit, split

shapefile = gpd.read_file(pathShapeF)
lista_zone = shapefile.FID.to_list()

intv_1='8H'
#intv_1 = '1H' if intv in ['2H', '3H', '4H', '5H', '6H', '7H', '8H', '9H','10H', '11H', '12H'] else intv
# Lower bound of time interval
casa = time_limits_inf_2(casa, intv_1)
print('dopo limite',casa.head(1))

index_zone = list(range(len(lista_zone)))           # List from 0 to -1 (zones)
map_zone = dict(list(zip(lista_zone, index_zone)))  # Map each element with its position number
    # Take-over calendar bounds
min = df.from_timedate_gmt.min()
max = df.to_timedate_gmt.max()
cal = ks.date_range(start='2013-04-17 10:00', end='2013-04-18 16:00', freq=intv_1)
#casa.to_timedate_gmt = ks.to_datetime(casa.to_timedate_gmt)
#casa.from_timedate_gmt = ks.to_datetime(casa.from_timedate_gmt)

casa = add_peso_column(casa, cal)
print(casa.head(1))
cal = cal.to_list()
list_matrix_tot = []
df_check = casa.to_spark().cache()
lista_zone_1 = spark.createDataFrame(lista_zone, IntegerType()).select(col('value').alias('from_zone_fid'))
lista_zone_2 = spark.createDataFrame(lista_zone, IntegerType()).select(col('value').alias('to_zone_fid'))
crossListe = lista_zone_1.crossJoin(lista_zone_2)
for j in range(0, len(cal) - 1):
    print('---ADJ---', j)
    print(cal[j])
    print(cal[j+1])
    dataframe_ridotto = df_check.filter(((df_check.from_timedate_gmt >= cal[j]) &
                                  (df_check.from_timedate_gmt < cal[j + 1]))|
                                    ((df_check.to_timedate_gmt >= cal[j]) &
                                           (df_check.to_timedate_gmt <= cal[j + 1])) |
                                    ((df_check.from_timedate_gmt < cal[j]) &
                                           (df_check.to_timedate_gmt > cal[j + 1])))

    #print(dataframe_ridotto.head())
    print('---SHAPEADJ---', j)
    df_spark = dataframe_ridotto#.to_spark()
    print(df_spark.show())
    df_spark = df_spark.withColumn("peso", df_spark["peso"].cast(LongType()))
    df_grouped = df_spark.groupBy('from_zone_fid', 'to_zone_fid').agg(sum('peso').alias('peso'))
    df_grouped = crossListe.join(df_grouped, on = ['from_zone_fid', 'to_zone_fid'], how='left').fillna(0)
    matrix = df_grouped.select('peso').to_numpy().values.reshape((len(lista_zone), len(lista_zone)))
    print(matrix)
    matrix = (matrix.T / matrix.sum(axis=1)).T
    matrix = np.nan_to_num(matrix)
    matrix = csr_matrix(matrix)
    list_matrix_tot += [matrix]
    del matrix
exit

/home/gpu2/.local/lib/python3.8/site-packages/databricks/koalas/namespace.py:1754: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  pd.date_range(


dopo limite 

22/06/15 12:03:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:03:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 1

   idtrajectory  idterm  from_zone_fid  to_zone_fid  tripdistance_m  triptime_s  stoptime_s   from_timedate_gmt     to_timedate_gmt
0     104214517    1432             -1         1006           29220       10262        5008 2013-04-17 11:00:00 2013-04-17 11:00:00


/home/gpu2/.local/lib/python3.8/site-packages/databricks/koalas/namespace.py:1754: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  pd.date_range(
22/06/15 12:04:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/

...

/home/gpu2/.local/lib/python3.8/site-packages/databricks/koalas/generic.py:603: UserWarning: We recommend using `DataFrame.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
22/06/15 12:04:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single

---ADJ--- 1
2013-04-17 18:00:00
2013-04-18 02:00:00
---SHAPEADJ--- 1
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+
|idtrajectory|idterm|from_zone_fid|to_zone_fid|tripdistance_m|triptime_s|stoptime_s|from_timedate_gmt|to_timedate_gmt|peso|
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+

None


/home/gpu2/.local/lib/python3.8/site-packages/databricks/koalas/generic.py:603: UserWarning: We recommend using `DataFrame.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
22/06/15 12:04:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:04:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single

---ADJ--- 2
2013-04-18 02:00:00
2013-04-18 10:00:00
---SHAPEADJ--- 2
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+
|idtrajectory|idterm|from_zone_fid|to_zone_fid|tripdistance_m|triptime_s|stoptime_s|from_timedate_gmt|to_timedate_gmt|peso|
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+

None


/home/gpu2/.local/lib/python3.8/site-packages/databricks/koalas/generic.py:603: UserWarning: We recommend using `DataFrame.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
22/06/15 12:05:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single

In [9]:
# Transiction

from pyspark.sql.window import Window
from pyspark.sql.functions import concat_ws, row_number, date_format,from_unixtime,lag, col, lead, to_date, unix_timestamp, datediff, lit, split
import timeit


intv='1H'

start = ks.to_datetime(casa.from_timedate_gmt.min(), format='%Y/%m/%d %H:%M')  # Lower date in the df
start = start.replace(minute=00)  # Set minutes to 00
end = ks.to_datetime(casa.to_timedate_gmt.max(), format='%Y/%m/%d %H:%M')  # Upper date in the df
end = end.replace(hour=23, minute=59)  # Upper date set to 23:59
calendar = ks.date_range(start='2013-04-17 13:00', end='2013-04-17 19:00', freq=intv)
cal = calendar.to_list()
cal = ks.to_datetime(cal)
casa = add_new_stoptime_column(casa)
print(casa.head(1))
shapefile = gpd.read_file(pathShapeF)
lista_zone = shapefile.FID.to_list()
index_zone = list(range(len(lista_zone)))  # List from 0 to num zones
map_zone = dict(list(zip(lista_zone, index_zone)))  # Map each element with its position number
list_matrix_tot = []
df_check = casa.to_spark().cache()
lista_zone_2 = spark.createDataFrame(lista_zone, IntegerType()).select(col('value').alias('to_zone_fid'))
print(type(df_check))
for j in range(0, len(cal) - 1):
    print('---FEAT---', j)
    print('calendar', cal[j])
    print('calendar', cal[j+1])
          
#   
          # Dataset reduction with each time interval
    #dataframe_ridotto = df_check.loc[((df_check.to_timedate_gmt >= cal[j]) &
    #                                  (df_check.to_timedate_gmt < cal[j + 1])) |
    #                                 ((df_check.totimedate_plus_stoptime > cal[j]) &
    #                                  (df_check.totimedate_plus_stoptime <= cal[j + 1])) |
    #                                 ((df_check.to_timedate_gmt < cal[j]) &
    #                                  (df_check.totimedate_plus_stoptime > cal[j + 1]))]
    dataframe_ridotto = df_check.filter(((df_check.to_timedate_gmt >= cal[j]) &
                                      (df_check.to_timedate_gmt < cal[j + 1])) |
                                     ((df_check.totimedate_plus_stoptime > cal[j]) &
                                      (df_check.totimedate_plus_stoptime <= cal[j + 1])) |
                                     ((df_check.to_timedate_gmt < cal[j]) &
                                      (df_check.totimedate_plus_stoptime > cal[j + 1])))
    #dataframe_ridotto = df_check.loc[((df_check.to_timedate_gmt >= cal[j]) &
#   
    #                                   (df_check.totimedate_plus_stoptime <= cal[j + 1]))]
#   
    #estraggo la prima riga per ogni id_term e id_pickup
    print('nuovooo \n')
    #print(dataframe_ridotto.head(4))
    #ataframe_ridotto.head(1) !=0:
    start_time = timeit.default_timer()
    #if len(dataframe_ridotto.head(1)) != 0:
    print('---SHAPEADJ---', j)
    df_spark = dataframe_ridotto #.to_spark()
    print(df_spark.show())
    df_grouped = df_spark.groupBy('to_zone_fid').agg(count('to_zone_fid').alias('count'))
    df_grouped = lista_zone_2.join(df_grouped, on = ['to_zone_fid'], how='left').fillna(0)
    matrix = df_grouped.select('count').to_koalas().values.reshape((len(lista_zone), 1))

    #matrix = df_grouped.select('count')
    #print(matrix.show())
    #print((matrix.count(), len(matrix.columns)))
    #matrix = np.concatenate(df_grouped.select('count').rdd.glom().map(lambda x: np.array([elem[0] for elem in x])).collect()).reshape((len(lista_zone), 1))
    #matrix = df_grouped.select('count').write.save(path='csv', format='csv', mode='append', sep='\t')
    #to_koalas().values.reshape((len(lista_zone), 1))
    #print(type(matrix))
    #Your statements here
    stop_time = timeit.default_timer()
    print('Time: ', stop_time - start_time)
    #print(matrix)
    matrix = csr_matrix(matrix)
    list_matrix_tot += [matrix]
    del matrix
exat
    #else:
    #    matrix = np.zeros((len(lista_zone), 1))
    #    matrix = csr_matrix(matrix)
    #    list_matrix_tot += [matrix]
    #    del matrix
    #    continue

/home/gpu2/.local/lib/python3.8/site-packages/databricks/koalas/namespace.py:1754: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  pd.date_range(
22/06/15 12:05:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


   idtrajectory  idterm  from_zone_fid  to_zone_fid  tripdistance_m  triptime_s  stoptime_s   from_timedate_gmt     to_timedate_gmt  peso totimedate_plus_stoptime
0     104214517    1432             -1         1006           29220       10262        5008 2013-04-17 11:00:00 2013-04-17 11:00:00   1.0      2013-04-17 12:24:00
<class 'pyspark.sql.dataframe.DataFrame'>
---FEAT--- 0
calendar 2013-04-17 13:00:00
calendar 2013-04-17 14:00:00
nuovooo 

---SHAPEADJ--- 0
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+------------------------+
|idtrajectory|idterm|from_zone_fid|to_zone_fid|tripdistance_m|triptime_s|stoptime_s|from_timedate_gmt|to_timedate_gmt|peso|totimedate_plus_stoptime|
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+------------------------+
+------------+------+-------------+-----------+--------------+----------+----------+---

/home/gpu2/.local/lib/python3.8/site-packages/databricks/koalas/generic.py:603: UserWarning: We recommend using `DataFrame.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
22/06/15 12:05:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single

Time:  0.8551868539998395
---FEAT--- 1
calendar 2013-04-17 14:00:00
calendar 2013-04-17 15:00:00
nuovooo 

---SHAPEADJ--- 1
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+------------------------+
|idtrajectory|idterm|from_zone_fid|to_zone_fid|tripdistance_m|triptime_s|stoptime_s|from_timedate_gmt|to_timedate_gmt|peso|totimedate_plus_stoptime|
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+------------------------+
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+------------------------+

None


/home/gpu2/.local/lib/python3.8/site-packages/databricks/koalas/generic.py:603: UserWarning: We recommend using `DataFrame.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
22/06/15 12:05:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single

...

/home/gpu2/.local/lib/python3.8/site-packages/databricks/koalas/generic.py:603: UserWarning: We recommend using `DataFrame.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
22/06/15 12:05:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single

Time:  0.6949614910008677
---FEAT--- 5
calendar 2013-04-17 18:00:00
calendar 2013-04-17 19:00:00
nuovooo 

---SHAPEADJ--- 5
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+------------------------+
|idtrajectory|idterm|from_zone_fid|to_zone_fid|tripdistance_m|triptime_s|stoptime_s|from_timedate_gmt|to_timedate_gmt|peso|totimedate_plus_stoptime|
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+------------------------+
+------------+------+-------------+-----------+--------------+----------+----------+-----------------+---------------+----+------------------------+

None


/home/gpu2/.local/lib/python3.8/site-packages/databricks/koalas/generic.py:603: UserWarning: We recommend using `DataFrame.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
22/06/15 12:05:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 12:05:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single

Time:  0.698523869001292


NameError: name 'exat' is not defined